In [1]:
!pip install groq
GROQ_API_KEY='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'
import os
from groq import Groq
client = Groq(
    api_key=GROQ_API_KEY,
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [4]:
def chatbot(l):
    # Start an empty conversation
    conversation = []
    b=[]
    # print("Chatbot: Hi! How can I help you today?")
    i=0
    while i<len(l):
        # Get input from the user
        # user_input = input("You: ")

        # Check if the user wants to quit
        if l[i].lower() in ["quit", "exit", "bye"]:
            # print("Chatbot: Goodbye!")
            break

        # Add user's message to the conversation
        conversation.append({"role": "user", "content": l[i]})

        # Get chatbot completion
        completion = client.chat.completions.create(
            model="llama3-8b-8192",  # This is the model you're using
            messages=conversation,   # Pass the conversation so far
            temperature=1,
            max_tokens=1024,
            top_p=1,
            stream=True,
            stop=None,
        )

        # Output chatbot's response
        bot_response = ""
        for chunk in completion:
            response_part = chunk.choices[0].delta.content or ""
            # print(response_part, end="")
            bot_response += response_part

        print()  # Print a newline after the response
        conversation.append({"role": "assistant", "content": bot_response})
        i=i+1
        b.append(bot_response)
    hsn=b[1:]
    hsn_1=list()
    for i in hsn:
      hsn_1.append(remove_dot(i))
    return hsn_1
l=["i will give you product discription you just give me HS code for that products if you confuse or multiple answers just give me most probable answer like:123346","Almonds"]
r=chatbot(l)

In [2]:
def remove_dot(a):
  l=""
  for i in a:
    if i in ['0','1','2','3','4','5','6','7','8','9']:
      l+=i
  return l

In [8]:
def remove_per(a):
  if a[0]==' ':
    return remove_per(a[1:])
  b=''
  for i in a:
    if i==' ' or i=='%':
      break
    else:
      b+=i
  return float(b)

In [5]:
def get_gst(food_description):
  lis=[]
  for food in food_description:
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"{food}. Give answer"
            }
        ],
        model="llama3-8b-8192",
    )
    # global msg
    msg=f"GST for '{food}': {chat_completion.choices[0].message.content}"
    l=msg.split(":")
    lis.append(l[1])
    l=lis[1:]
    list1=[]
  for i in l:
    list1.append(remove_per(i))
  return list1
l=['I will give you food description of foods and you just tell me only numerical value of GST on that in India of 2023 GST Model if you give me value you can not change it',
 'give me GST rate in percentage for Oil only numerical value',
 'give me GST rate in percentage for Fruits Juice only numerical value']

In [6]:
import pandas as pd
def gst_calculate():
  gst_descriptions=["I will give you food description of foods and you just tell me only numerical value of GST on that in India" ]
  # hsn_descriptions=['I will give you food description of foods and you just tell me only HSN Code on that in India:']
  hsn=["i will give you product discription you just give me HS code for that products if you confuse or multiple answers just give me most probable answer like:123346"]
  description=[]
  Rate_without_gst=[]
  Rate_with_gst=[]
  no_of_quantity=[]
  total_without_gst=0
  total_with_gst=0
  n=int(input('Enter Number of items: '))
  for i in range(n):
    description_of_food=input('Enter the Discription of food: ')
    number_of_quan=int(input('Enter no. of Quantity'))
    rate_without_gst=float(input('Enter rate without GST: '))
    description.append(description_of_food)
    hsn.append(description_of_food)
    Rate_without_gst.append(rate_without_gst)
    no_of_quantity.append(number_of_quan)
    gst_descriptions.append(f"give me GST rate in percentage for {description_of_food} only numerical value")
    # hsn_descriptions.append(f"give me HSN code for {description_of_food}")
  lis=get_gst(gst_descriptions)
  # hsn=get_HSN(hsn_descriptions)
  for i in range(n):
    Rate_with_gst.append(Rate_without_gst[i]*(lis[i]+100)/100)
  description.append("Total")
  Rate_without_gst.append(None)
  Rate_with_gst.append(None)
  lis.append(None)
  l=chatbot(hsn)
  l.append(None)
  # l.append(None)
  no_of_quantity.append(None)
  total_rate_without_gst=[]
  total_rate_with_gst=[]
  for i in range(len(Rate_without_gst)-1):
    total_rate_without_gst.append(no_of_quantity[i]*Rate_without_gst[i])
    total_rate_with_gst.append(no_of_quantity[i]*Rate_with_gst[i])
  total1=0
  total2=0
  for i in range(len(no_of_quantity)-1):
    total1+=no_of_quantity[i]*Rate_without_gst[i]
    total2+=no_of_quantity[i]*Rate_with_gst[i]
  total_rate_with_gst.append(total2)
  total_rate_without_gst.append(total1)
  l={"Description of Foods":description,"HSN Code":l,"No.of Quantity":no_of_quantity,"Rate without GST per kg\litre":Rate_without_gst,"Rate_with_GST per kg\litre":Rate_with_gst,"GST_percentage":lis,"Total Rate without GST":total_rate_without_gst,"Total Rate with GST":total_rate_with_gst}
  d=pd.DataFrame(l)
  print(d)

In [9]:
gst_calculate()

Enter Number of items: 1
Enter the Discription of food: Juice
Enter no. of Quantity2
Enter rate without GST: 50


  Description of Foods HSN Code  No.of Quantity  \
0                Juice       20             2.0   
1                Total     None             NaN   

   Rate without GST per kg\litre  Rate_with_GST per kg\litre  GST_percentage  \
0                           50.0                        56.0            12.0   
1                            NaN                         NaN             NaN   

   Total Rate without GST  Total Rate with GST  
0                   100.0                112.0  
1                   100.0                112.0  
